In [5]:
# Load packages
pacman::p_load(dplyr, stringr, data.table, tidyr, data.table, Matrix,
               hdf5r, Seurat, Signac,harmony, knitr, SoupX, cluster, glmgampoi,
               EnsDb.Hsapiens.v86, 
               logr, parallel, future,
               fpc,
               ggplot2, ggpubr, ggrepel, ggbreak, gridExtra, patchwork, grid, ggh4x)

Warning message:
“package ‘glmgampoi’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Warning message in p_install(package, character.only = TRUE, ...):
“”
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘glmgampoi’”
Warning message in pacman::p_load(dplyr, stringr, data.table, tidyr, data.table, :
"Failed to install/load:
glmgampoi"


# Process GWAS summarys stats

In [ ]:
'Summary stats from other studies are usually a mess, there is some pre-processing to be done'
'I created a google sheet where I report with unique IDs the study: '

'https://docs.google.com/spreadsheets/d/1CEY6lpp3-KTTyUwgHAQo7Tmei7AFmEs3TfQcwZpQM_k/edit?usp=sharing'


  -h, --help            show this help message and exit
  --sumstats SUMSTATS   Input filename.
  --N N                 Sample size If this option is not set, will try to
                        infer the sample size from the input file. If the
                        input file contains a sample size column, and this
                        flag is set, the argument to this flag has priority.
  --N-cas N_CAS         Number of cases. If this option is not set, will try
                        to infer the number of cases from the input file. If
                        the input file contains a number of cases column, and
                        this flag is set, the argument to this flag has
                        priority.
  --N-con N_CON         Number of controls. If this option is not set, will
                        try to infer the number of controls from the input
                        file. If the input file contains a number of controls
                        column, and this flag is set, the argument to this
                        flag has priority.
  --out OUT             Output filename prefix.
  --info-min INFO_MIN   Minimum INFO score.
  --maf-min MAF_MIN     Minimum MAF.
  --daner               Use this flag to parse Stephan Ripke's daner* file
                        format.
  --daner-n             Use this flag to parse more recent daner* formatted
                        files, which include sample size column 'Nca' and
                        'Nco'.
  --no-alleles          Don't require alleles. Useful if only unsigned summary
                        statistics are available and the goal is h2 /
                        partitioned h2 estimation rather than rg estimation.
  --merge-alleles MERGE_ALLELES
                        Same as --merge, except the file should have three
                        columns: SNP, A1, A2, and all alleles will be matched
                        to the --merge-alleles file alleles.
  --n-min N_MIN         Minimum N (sample size). Default is (90th percentile
                        N) / 2.
  --chunksize CHUNKSIZE
                        Chunksize.
  --snp SNP             Name of SNP column (if not a name that ldsc
                        understands). NB: case insensitive.
  --N-col N_COL         Name of N column (if not a name that ldsc
                        understands). NB: case insensitive.
  --N-cas-col N_CAS_COL
                        Name of N column (if not a name that ldsc
                        understands). NB: case insensitive.
  --N-con-col N_CON_COL
                        Name of N column (if not a name that ldsc
                        understands). NB: case insensitive.
  --a1 A1               Name of A1 column (if not a name that ldsc
                        understands). NB: case insensitive.
  --a2 A2               Name of A2 column (if not a name that ldsc
                        understands). NB: case insensitive.
  --p P                 Name of p-value column (if not a name that ldsc
                        understands). NB: case insensitive.
  --frq FRQ             Name of FRQ or MAF column (if not a name that ldsc
                        understands). NB: case insensitive.
  --signed-sumstats SIGNED_SUMSTATS
                        Name of signed sumstat column, comma null value (e.g.,
                        Z,0 or OR,1). NB: case insensitive.
  --info INFO           Name of INFO column (if not a name that ldsc
                        understands). NB: case insensitive.
  --info-list INFO_LIST
                        Comma-separated list of INFO columns. Will filter on
                        the mean. NB: case insensitive.
  --nstudy NSTUDY       Name of NSTUDY column (if not a name that ldsc
                        understands). NB: case insensitive.
  --nstudy-min NSTUDY_MIN
                        Minimum # of studies. Default is to remove everything
                        below the max, unless there is an N column, in which
                        case do nothing.
  --ignore IGNORE       Comma-separated list of column names to ignore.
  --a1-inc              A1 is the increasing allele.
  --keep-maf            Keep the MAF column (if one exists).

In [ ]:
# 1 trait per folder

# Unzip them
'yes, u have to manually figure out if they are gzip, unzip or whatever, click on the link and see'
# unzip or gunzip or tar

# Fix the n (add the column and make sure all tables have this format:/nfs/lab/Luca/Assets/GWAS/CVDKP/CVDKP_7/CAD_GWAS_BBJ_meta_w_hm_rsids.tsv)

# Add rsids

# Munge them into traits location

## MVP Munging - luca

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/CAD.EUR.MVP_UKBB_cardC4D.NatMed2022.txt.gz \
--a1 EA \
--a2 NEA \
--p pvalue \
--snp rsmid \
--N-col TotalSampleSize \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats beta,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/EUR.MVP_UKBB_cardC4D.NatMed2022.ldsc

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/CAS.EUR.MVP.Circulation2023.txt.gz \
--a1 A1 \
--a2 A2 \
--ignore EA \
--p PVAL \
--snp SNP_ID \
--N 744995 \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats beta,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/CAS.EUR.MVP.Circulation2023.ldsc

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/CLEANED.MVP.EUR.PAD.results.anno.nodup.txt.gz \
--a1 EFFECT_ALLELE \
--a2 OTHER_ALLELE \
--p PVAL \
--snp SNP_ID \
--N-col N_TOTAL \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats beta,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/CLEANED.MVP.EUR.PAD.ldsc

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/DBP_MVP_White.results.gz \
--a1 Allele1 \
--a2 Allele2 \
--ignore EA \
--p Pvalue \
--snp SNP_ID \
--N-col SampleSize \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats EffectEstimate,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/DBP_MVP_White.ldsc

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/HF.EUR.MVP.NatComm2022.txt.gz \
--a1 Allele_1 \
--a2 Allele_2 \
--ignore Effect_allele \
--p P_value \
--snp SNP_ID \
--N-col Sample_size \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats Estimate_Effect,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/HF.EUR.MVP.NatComm2022.ldsc

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/HFpEF.EUR.MVP.NatComm2022.txt.gz \
--a1 Allele_1 \
--a2 Allele_2 \
--ignore Effect_allele \
--p P_value \
--snp SNP_ID \
--N-col Sample_size \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats Estimate_Effect,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/HFpEF.EUR.MVP.NatComm2022.ldsc

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/HFrEF.EUR.MVP.NatComm2022.txt.gz \
--a1 Allele_1 \
--a2 Allele_2 \
--ignore Effect_allele \
--p P_value \
--snp SNP_ID \
--N-col Sample_size \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats Estimate_Effect,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/HFrEF.EUR.MVP.NatComm2022.ldsc

In [ ]:
##############PROBLEMATIC ONE #######################

In [52]:
## This i have to logn the OR, there is no beta
data = fread("/nfs/lab/MVP/MVP.EUR.AAA.fordbGaP.txt.gz")
head(data)

SNP_ID,CHROM,POS,Allele1,Allele2,EA,EAF,N,N_CASE,N_CONTROL,OR,CI,SE,P,IMP_QUAL
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
rs144804129,10,100000122,T,A,T,0.9997275,144970,5002,139968,1.0880205,"0.1409, 8.403",1.04300,0.93490,0.433755
rs6602381,10,10000018,A,G,A,0.5589000,144970,5002,139968,0.9819057,"0.9425, 1.023",0.02092,0.38280,0.985864
rs11442554,10,100000554,I,D,I,0.4884000,144970,5002,139968,1.0094554,"0.9557, 1.066",0.02791,0.73590,0.545618
rs7899632,10,100000625,A,G,A,0.5670000,144970,5002,139968,1.0000851,"0.9601, 1.042",0.02082,0.99670,0.999831
rs61875309,10,100000645,A,C,A,0.7978000,144970,5002,139968,1.0012698,"0.951, 1.054",0.02629,0.96150,0.959398
rs574200131,10,100001724,G,A,G,0.9986600,144970,5002,139968,0.5658649,"0.3178, 1.008",0.29440,0.07128,0.577888


In [54]:
# Add Beta
data$BETA = log(data$OR)

In [56]:
# Specify the file name
file_name <- "/nfs/lab/Luca/Assets/GWAS/MVP/MVP.EUR.AAA.fordbGaP_BETA.gz"

# Save the data frame to a gzipped file
write.table(data, file = gzfile(file_name), row.names = FALSE, quote = FALSE, sep = "\t")

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/Luca/Assets/GWAS/MVP/MVP.EUR.AAA.fordbGaP_BETA.gz \
--a1 Allele1 \
--a2 Allele2 \
--p P \
--snp SNP_ID \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats BETA,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/MVP.EUR.AAA.fordbGaP.ldsc

In [ ]:
##############PROBLEMATIC ONE #######################

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/MVP.T2D.EUR.MAF001.dbGaP.checked.txt.gz \
--a1 EA \
--a2 NEA \
--ignore EAF \
--p P \
--snp RSID \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats BETA,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/MVP.T2D.EUR.MAF001.dbGaP.ldsc

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/PP_MVP_White.results.gz \
--a1 Allele1 \
--a2 Allele2 \
--ignore EA \
--p Pvalue \
--snp SNP_ID \
--N-col SampleSize \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats EffectEstimate,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/PP_MVP_White.ldsc

In [ ]:
python /nfs/lab/ldsc/munge_sumstats.py \
--sumstats /nfs/lab/MVP/SBP_MVP_White.results.gz \
--a1 Allele1 \
--a2 Allele2 \
--ignore EA \
--p Pvalue \
--snp SNP_ID \
--N-col SampleSize \
--chunksize 500000 \
--frq af_alt \
--maf-min 0.01 \
--signed-sumstats EffectEstimate,0 \
--merge-allele /nfs/lab/relgamal/LDSC/w_hm3.snplist \
--out /nfs/lab/Luca/Assets/GWAS/MVP/SBP_MVP_White.results.ldsc